# LLM-powered AI Agents

Table of contents
1. Understanding LLMs
2. Tools
3. Chat-based AI Agents
4. Service-based AI agents
5. Multi-Agents

In [ ]:
import json
import pandas as pd
import random
import uvicorn
from pathlib import Path
from datetime import datetime
from sklearn.metrics import accuracy_score
from pydantic import BaseModel, Field
from typing import Any
from fastapi import FastAPI
from io import StringIO
from language_models.agents.chain import AgentChain
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole
from language_models.tools.tool import Tool
from language_models.proxy_client import BTPProxyClient
from language_models.agents.react import ReActAgent
from language_models.tools.earthquake import earthquake_tools
from language_models.tools.current_date import current_date_tool
from language_models.settings import settings
from pprint import pprint

In [ ]:
proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## 1. Understanding LLMs

In [ ]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-35-turbo",
    max_tokens=256,
    temperature=0.0,
)

In [ ]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!"""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

In [ ]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!

Respond with positive or negative."""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

In [ ]:
system_prompt = "Take the following movie review and determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Wow! This movie was incredible. The acting was superb, and the plot kept me on the edge of my seat. I highly recommend it!"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

In [ ]:
system_prompt = "Take the following movie review determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
pprint(response)

In [ ]:
system_prompt = """Take the following movie review and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a movie review, respond with -1."""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

## 2. Tools

In [ ]:
prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12" # answer: $1,289.98

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

In [ ]:
def calculator(expression: str) -> Any:
    return eval(expression)

class Calculator(BaseModel):
    expression: str = Field(description="A math expression.")

calculator_tool = Tool(
    func=calculator,
    name="Calculator",
    description="Use this tool when you want to do calculations.",
    args_schema=Calculator
)

print(calculator_tool)

In [ ]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Valid "tool" values: {calculator_tool.name}

Always use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use",
  "tool_input": "Valid key value pairs",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

In [ ]:
print(calculator(**response["tool_input"]))

In [ ]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Valid "tool" values: {calculator_tool.name}

Always use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use",
  "tool_input": "Valid key value pairs",
}}

Observation: tool result
... (this Thought/Tool/Observation can repeat N times)

When you know the answer, use the following JSON format:
{{
  "thought": "I now know what to respond",
  "tool": "Final Answer",
  "tool_input": "The final answer to the question",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=json.dumps(response)),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=f"Response of Calculator tool: {calculator(**response['tool_input'])}"),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

## 3. Chat-based AI Agents

### Earthquake

In [ ]:
system_prompt = """You are an United States Geological Survey expert who can answer questions regarding earthquakes and can run forecasts.

Use the current date tool to access the local date and time before using other tools.

Take the following question and answer it as accurately as possible."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4-32k',
    max_tokens=2048,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

earthquake_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{question}",
    task_prompt_variables=["question"],
    tools=earthquake_tools + [current_date_tool],
    output_format=Output,
    iterations=10,
)

In [ ]:
response = earthquake_agent.invoke({"question": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"})

In [ ]:
print(response.final_answer["content"])

In [ ]:
response = earthquake_agent.invoke({"question": "Query 10 earthquakes that occurred yesterday and have a magnitude > 3."})

In [ ]:
print(response.final_answer["content"])

In [ ]:
response = earthquake_agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

In [ ]:
pprint(response.final_answer["content"])

## 4. Service-based AI Agents

### Contract Drafting

In [ ]:
system_prompt = """Take the follow content and generate a draft of a section for a contract.

Write the content from scratch and make it sound professional.

Respond with the title of the section and the content of the section."""

task_prompt = """Section content:
{section}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class ContractSection(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")

contract_drafting_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["section", "section_name"],
    tools=None,
    output_format=ContractSection,
    iterations=5,
)

In [ ]:
def generate_contract(contract_sections: list[str]) -> str:
    sections = []
    for contract_section in contract_sections:
        response = contract_drafting_agent.invoke({"section": contract_section})
        section = response.final_answer["title"] + "\n\n" + response.final_answer["content"]
        sections.append(section)
    return "\n\n".join(sections)

In [ ]:
definitions = "Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA."

amendment = """INVOICING AND PAYMENT TERMS
Clause 12.1(ii) of the GMA shall be cancelled and substituted as follow:
[*****]
[*****]
[*****]

Any other provision of Clause 12 shall remain in full force and effect.

PRICE CONDITIONS
(i) Clause 3.2 of the Exhibit 14 of the GMA shall be cancelled and substituted as follow:
“3.2 Technical conditions for prices adjustment
The prices set out in this Exhibit 14 shall be modified every [*****] at the occasion of the invoicing reconciliation pursuant to Clause 11
(“Reconciliation”) if the Standard Operations of the Aircraft, analyzed at the time of the adjustment (all calculations are made with figures corresponding to [*****], change by more or less
[*****] with respect to the estimated values of the same parameters, considered at the time of commencement of the Term.
As from the date this Agreement enters into force, the Parties agree to take into account the following basic operating parameters (the
“Standard Operations”) as a reference for the above calculation:
[*****]
[*****]
[*****]"""

effective_date_and_duration = "Amendment is effective starting on the date of its signature by both Parties."

confidentiality = """Confidential Information released by either of the Parties (the “Disclosing Party”) to the
other Party (the “Receiving Party”) shall not be released in whole or in part to any third party:
- Not to deliver, disclose or publish it to any third party including subsidiary companies and companies having an interest in its capital
- Use Confidential Information solely for the purpose of this Amendment
- Disclose the Confidential Information only to those of its direct employees
- Not to duplicate the Confidential Information nor to copy

Any Confidential Information shall remain the property of the Disclosing Party.

The Receiving Party hereby acknowledges and recognises that Confidential Information is protected by copyright Laws and related
international treaty provisions, as the case may be.

This shall survive termination or expiry of this Amendment for a period of five (5) years following such End Date."""

governing_law = """Pursuant to and in accordance with Section 5-1401 of the New York General Obligations Law.

Arbitration: in the event of a dispute arising out of or relating to this Amendment, including without limitation disputes regarding the
existence, validity or termination of this Amendment (a “Dispute”), either Party may notify such Dispute to the other through service of a
written notice (the “Notice of Dispute”).

Arbitration, and any proceedings, and meetings incidental to or related to the arbitration process, shall take place in New York.

Arbitration shall be kept confidential and the existence of the proceeding and any element.

During any period of negotiation or arbitration, the Parties shall continue to meet their respective obligations.

Notwithstanding any provision of this the Parties may, at any time, seek and decide to settle a Dispute.

Judgment upon any award may be entered in any court having jurisdiction.

Recourse to jurisdictions is expressly excluded except as provided for in the ICC Rules of Conciliation and Arbitration."""

miscellaneous = """Amendment contains the entire agreement between the Parties regarding the subject-matter.

Amendment shall not be varied or modified except by a written document duly signed."""

contract = generate_contract(
    contract_sections=[
        definitions,
        amendment,
        effective_date_and_duration,
        confidentiality,
        governing_law,
        miscellaneous,
    ]
)

In [ ]:
print(contract)

### Sentiment Analysis

In [ ]:
df_tweets = pd.read_csv("./data/tweets.csv.gz", compression="gzip", encoding="latin-1", names=["sentiment", "id", "date", "query", "user", "tweet"])
df_tweets = df_tweets.dropna()
df_tweets = df_tweets.where(df_tweets.sentiment != 2)
df_tweets["sentiment"] = df_tweets["sentiment"].map({4: 1, 0: 0})
df_tweets_sampled = df_tweets.sample(n=10)
df_tweets_sampled.head()

In [ ]:
system_prompt = """Take the following tweet and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a tweet, respond with -1."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class Sentiment(BaseModel):
    sentiment: int = Field(description="The sentiment of the tweet.")

sentiment_analysis_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Tweet:\n{tweet}",
    task_prompt_variables=["tweet"],
    tools=None,
    output_format=Sentiment,
    iterations=2,
)

In [ ]:
def classify_sentiment(tweet: str) -> int:
    response = sentiment_analysis_agent.invoke({'tweet': tweet})
    return response.final_answer['sentiment'] or 0

In [ ]:
df_tweets_sampled["prediction"] = [classify_sentiment(tweet) for tweet in df_tweets_sampled.tweet]

In [ ]:
print(f"Accuracy: {accuracy_score(df_tweets_sampled.sentiment, df_tweets_sampled.prediction)}")

### Structuring Unstructured Data

In [ ]:
path = Path("./data/jobs")
filenames = [file.name for file in path.iterdir() if file.is_file()]
filenames = random.sample(filenames, 5)

jobs = []
for filename in filenames:
    file_path = path / filename
    with open(file_path, "r", encoding="utf-8", errors="replace") as file:
        content = file.read()
        jobs.append(content)

In [ ]:
system_prompt = """Take the following job and extract data about the job.

Respond with the following extracted data:
- job_title: The job title.
- job_class_no: The job class code.
- job_duties: The duties of the job.
- open_date: When the position was opened. Format: DD-MM-YYYY.
- salary: The salary ranges. Format: 'min salary to max salary'.
- deadline: The application deadline. Format: DD-MM-YYYY.
- application_form: The form of the application (e.g. online, fax, email).
- where_to_apply: The url to apply at or location to send the fax or email address."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=2048,
    float=0.0,
)

class Job(BaseModel):
    job_title: str = Field(description="The job title.")
    job_class_no: int = Field(description="The job class code.")
    job_duties: str = Field(description="The duties of the job.")
    open_date: str = Field(description="When the position was opened. Format: DD-MM-YYYY.")
    salary: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")
    deadline: str = Field(description="The application deadline. Format: DD-MM-YYYY")
    application_form: str = Field(description="The form of the application (e.g. online, fax, email).")
    where_to_apply: str = Field(description="The url to apply at or location to send the fax or email address.")

job_data_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job}",
    task_prompt_variables=["job"],
    tools=None,
    output_format=Job,
    iterations=10,
)

In [ ]:
def extract_jobs(jobs: list[str]) -> pd.DataFrame:
    data = []
    for job in jobs:
        response = job_data_agent.invoke({"job": job})
        data.append(response.final_answer)
    return pd.DataFrame(data)

In [ ]:
df_jobs = extract_jobs(jobs)

In [ ]:
df_jobs.head()

## 5. Multi-Agents

### Comparing Unstructured Data

In [ ]:
def get_job(path: str) -> str:
    with open(path, "r", encoding="utf-8") as file:
        content = file.read()
        return content

job1 = get_job("./data/jobs/ELECTRICAL ENGINEERING ASSOCIATE 7525 093016 REV 100416.txt")
job2 = get_job("./data/jobs/ELECTRICAL MECHANIC 3841 012017.txt")

In [ ]:
system_prompt = """Take the following job and extract data about the job.

Respond with the following extracted data:
- job_title: The job title.
- job_duties: The duties of the job.
- salary: The salary ranges. Format: 'min salary to max salary'."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class Job1(BaseModel):
    job1_title: str = Field(description="The job title.")
    job1_duties: str = Field(description="The duties of the job.")
    salary1: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

class Job2(BaseModel):
    job2_title: str = Field(description="The job title.")
    job2_duties: str = Field(description="The duties of the job.")
    salary2: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

job_agent1 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job1}",
    task_prompt_variables=["job1"],
    tools=None,
    output_format=Job1,
    iterations=10,
)

job_agent2 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job2}",
    task_prompt_variables=["job2"],
    tools=None,
    output_format=Job2,
    iterations=10,
)

In [ ]:
system_prompt = "Take the following 2 job descriptions and respond with the similarities and differences of the jobs."

task_prompt = """Compare the 2 given job descriptions:

Job 1:
Job title: {job1_title}
Job duties:
{job1_duties}
Salary:
{salary1}


Job 2:
Job title: {job2_title}
Job duties:
{job2_duties}
Salary:
{salary2}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class JobComparison(BaseModel):
    similarities: str = Field(description="The job similarities.")
    differences: str = Field(description="The job differences.")

job_comparison_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["job1_title", "job1_duties", "salary1", "job2_title", "job2_duties", "salary2"],
    tools=None,
    output_format=JobComparison,
    iterations=10,
)

In [ ]:
chain = AgentChain(
    chain=[job_agent1, job_agent2, job_comparison_agent],
    chain_variables=["job1", "job2"],
)

In [ ]:
response = chain.invoke({"job1": job1, "job2": job2})

In [ ]:
pprint(response.final_answer["similarities"])

In [ ]:
pprint(response.final_answer["differences"])

### Machine Learning Code Generation

In [ ]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

Respond with 1 of the following machine learning problems:
- Classification
- Regression
- Clustering
- Time series forecasting"""

task_prompt = """Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
{problem_description}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class ModelingProblem(BaseModel):
    modeling_problem: str = Field(description="The machine learning problem.")

problem_finder_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["problem_description", "dataset_size", "dataset_schema"],
    tools=None,
    output_format=ModelingProblem,
    iterations=5,
)

In [ ]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

You can solve machine learning problems for:
- Classification
- Regression
- Clustering
- Time series forecasting

You have access to the following Python libraries:
- pandas
- numpy
- scikit-learn"""

task_prompt = """Given the following machine learning problem, respond with Python code.

Modeling problem: {modeling_problem}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}
First 10 rows of dataset:
{dataset_snippet}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=2048,
    float=0.0,
)

class AutoMLCode(BaseModel):
    code: str = Field(description="The Python machine learning code.")

ml_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["modeling_problem", "dataset_size", "dataset_schema", "dataset_snippet"],
    tools=None,
    output_format=AutoMLCode,
    iterations=10,
)

In [ ]:
ml_chain = AgentChain(
    chain=[problem_finder_agent, ml_agent],
    chain_variables=["problem_description", "dataset_size", "dataset_schema", "dataset_snippet"]
)

In [ ]:
info_str = StringIO()
df_tweets.info(buf=info_str)
dataset_schema = info_str.getvalue()

In [ ]:
response = ml_chain.invoke({
    "problem_description": "I want to classify the sentiment of tweets.",
    "dataset_size": len(df_tweets),
    "dataset_schema": dataset_schema,
    "dataset_snippet": str(df_tweets.head(10).to_markdown())
})

In [ ]:
print(response.final_answer["code"])

### Forecasting

In [ ]:
from datetime import timedelta
from language_models.tools.forecasting import get_earthquakes_data, ml_model

In [ ]:
class Forecast(BaseModel):
    start_time: str = Field(None, description='Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')
    end_time: str = Field(None, description='Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')

def forecast(start_time = None, end_time = None):
    if start_time is None:
        start_time = (datetime.now() - timedelta(days=30)).date()
    if end_time is None:
        end_time = (datetime.now().date())
    df = get_earthquakes_data('https://earthquake.usgs.gov/fdsnws/event/1/query?', start_time, end_time)
    df_pred = ml_model.predict(df)
    return {'predictions': df_pred.to_dict(orient='records')}

In [ ]:
forecasting_tool = Tool(func=forecast, name='forecast', description='Test forecast model on real-time events.', args_schema=Forecast)

In [ ]:
task_prompt = """Take the following question and determine the start and end time to respond with.

Question:
{question}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=256,
    float=0.0,
)

class DateRange(BaseModel):
    start_time: str = Field(description="Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")
    end_time: str = Field(description="Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")

time_wizard_agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt=task_prompt,
    task_prompt_variables=["question"],
    tools=[current_date_tool],
    output_format=DateRange,
    iterations=5,
)

In [ ]:
forecast_chain = AgentChain(
    chain=[time_wizard_agent, forecasting_tool],
    chain_variables=["question"],
)

In [ ]:
response = forecast_chain.invoke({"question": "Run a forecast using the past week as data."})

In [ ]:
print(response.final_answer["forecast"])

In [ ]:
app = FastAPI()

class Question(BaseModel):
    content: str

class Forecast(BaseModel):
    predictions: list[dict]

@app.get("/forecast")
def forecast(question: Question) -> Forecast:
    response = chain.invoke({"question": question.content})
    return response.final_answer["forecast"]

In [ ]:
if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

### LLM-backed Tools

In [ ]:
earthquake_agent.reset()
problem_finder_agent.reset()
ml_agent.reset()

In [ ]:
class EarthquakeAgent(BaseModel):
    question: str = Field(description="The question regarding earthquakes.")

def answer_earthquake_questions(question: str) -> Any:
    response = earthquake_agent.invoke({"question": question})
    return response.final_answer

earthquake_agent_tool = Tool(
    func=answer_earthquake_questions,
    name="Earthquake Agent",
    description="Use this tool to answer questions about earthquakes.",
    args_schema=EarthquakeAgent,
)

class MLAgent(BaseModel):
    problem_description: str = Field(description="The user problem.")
    dataset_size: int = Field(description="The size of the dataset."),
    dataset_schema: str = Field(description="The dataset schema or information."),
    dataset_snippet: str = Field(description="The dataset snippet aka the first couple of rows of the dataset.")

def generate_ml_code(problem_description: str, dataset_size: int, dataset_schema: str, dataset_snippet: str) -> Any:
    response = ml_chain.invoke({
        "problem_description": problem_description,
        "dataset_size": dataset_size,
        "dataset_schema": dataset_schema,
        "dataset_snippet": dataset_snippet,
    })
    return response.final_answer

ml_agent_tool = Tool(
    func=generate_ml_code,
    name="ML Agent",
    description="Use this tool to generate machine learning code given a problem.",
    args_schema=MLAgent,
)

In [ ]:
system_prompt = """You are an Agent that delegates tasks to other Agents by using the appropriate tools.

Use the Earthquake Agent when the question is about earthquakes.

Use the ML Agent when the user wants you to generate machine learning code."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4-32k',
    max_tokens=4096,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

almighty_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{prompt}",
    task_prompt_variables=["prompt"],
    tools=[earthquake_agent_tool, ml_agent_tool],
    output_format=Output,
    iterations=10,
)

In [ ]:
response = almighty_agent.invoke({"prompt": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"})

In [ ]:
print(response.final_answer["content"])

In [ ]:
prompt = f"""Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: {len(df_tweets)}
Schema:
{dataset_schema}
First 10 rows of dataset:
{df_tweets.head(10).to_markdown()}"""

response = almighty_agent.invoke({"prompt": prompt})

In [ ]:
print(response.final_answer["content"])